<a href="https://colab.research.google.com/github/hihi5456/pytorch/blob/main/testblock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install adabelief-pytorch
!pip install timm

     |████████████████████████████████| 296kB 6.1MB/s 


In [2]:
import torchvision
from torchvision import transforms, datasets

import torch.utils.data.dataloader
import matplotlib.pyplot as plt


import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,ConcatDataset
import adabelief_pytorch
import numpy  as np

from sklearn.metrics import roc_auc_score, auc


In [3]:
from timm.data import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD, IMAGENET_INCEPTION_MEAN, IMAGENET_INCEPTION_STD
from timm.models.efficientnet_blocks import round_channels, resolve_bn_args, resolve_act_layer, BN_EPS_TF_DEFAULT
from timm.models.efficientnet_builder import EfficientNetBuilder, decode_arch_def, efficientnet_init_weights
from timm.models.features import FeatureInfo, FeatureHooks
from timm.models.helpers import build_model_with_cfg, default_cfg_for_features
from timm.models.layers import create_conv2d, create_classifier, DropPath, to_2tuple, trunc_normal_
from timm.models.registry import register_model
import timm
import timm.models.efficientnet

In [6]:
!nvidia-smi


Tue Apr  6 05:30:07 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    28W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(456)
if device =='cuda':
    torch.cuda.manual_seed_all(456)

In [5]:
class PatchEmbed(nn.Module):
    """ Image to Patch Embedding
    """
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        img_size = to_2tuple(img_size)
        patch_size = to_2tuple(patch_size)
        num_patches = (img_size[1] // patch_size[1]) * (img_size[0] // patch_size[0])
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = num_patches

        self.proj = nn.Conv2d(in_chans, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        B, C, H, W = x.shape  
        # FIXME look at relaxing size constraints
        assert H == self.img_size[0] and W == self.img_size[1], \
            f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]})."
        x = self.proj(x).flatten(2).transpose(1, 2)
        x=x.reshape(len(x),self.num_patches*3,self.patch_size[0],self.patch_size[1])
        #print(x.shape)
        return x


In [6]:
class skipblock(nn.Module):
  def __init__(self, in_channel,out_channel,outsize):
    super(skipblock, self).__init__()
    
    
    self.conv0=nn.Conv2d(in_channel,out_channel,1)
    self.silu=nn.SiLU()
    
    self.block=nn.Sequential(
                    
            nn.Conv2d(out_channel, 64*out_channel, 1),
            nn.BatchNorm2d(out_channel*64),         
            nn.SiLU(),
            nn.Dropout2d(),
            

            nn.Conv2d(out_channel*64, out_channel*64, 3, 1, 1),
            nn.BatchNorm2d(out_channel*64),          
            nn.SiLU(),
            nn.Dropout2d(),

            nn.Conv2d(out_channel*64,out_channel,1),
            nn.BatchNorm2d(out_channel),
            nn.SiLU(),
            nn.Dropout2d(),
            
        )
    
    self.bn=nn.BatchNorm2d(out_channel)
    self.pool=nn.AdaptiveAvgPool2d(outsize)

  def forward(self, x):

    out=self.conv0(x)
    out=self.silu(out)
    nn.Dropout2d()
    skip=out
    #print(out.shape)
    out=self.block(out)
    #print(out.shape)
    out=out+skip
    #print(out.shape)
    out=self.bn(out)
    out=self.pool(out)

    return out

In [7]:
#gelu=nn.GELU()
class testblock2(nn.Module):
    def __init__(self,ni,num_class):
        super(testblock2, self).__init__()

        self.embeding=PatchEmbed(1500,100,3,100*100*3)

        
        self.downsample=skipblock(675,64,80)

  
        self.block0=skipblock(64,32,80)
        
        self.block1=skipblock(32,3,50)
        
        self.block2=skipblock(3,1,10)
        
     
        self.classifier = nn.Sequential(
            nn.Linear(1*10*10,num_class),
            
        )

    def forward(self,x):
        out=self.embeding(x)
        out=self.downsample(out)
        out=self.block0(out)
        out=self.block1(out)
        out=self.block2(out)


        out = out.view(out.size(0),-1)
        out=self.classifier(out)
        
        return out


In [8]:
model=testblock2(3,39)
#model=torch.nn.DataParallel(model)
model=model.to(device)

In [13]:
x=torch.rand((1,3,1500,1500)).to(device)
model(x)


tensor([[-0.0662,  0.0834,  0.2586,  0.0372, -0.3026,  0.3635, -0.0498,  0.2286,
         -0.2595,  0.0895, -0.0852, -0.2213,  0.0255,  0.1662,  0.2744,  0.0189,
         -0.2645, -0.4103,  0.0852, -0.1068, -0.0485,  0.3315,  0.3054, -0.0933,
         -0.1741,  0.0458,  0.1728,  0.1702,  0.0914, -0.1630, -0.1520,  0.0795,
         -0.1396, -0.1871, -0.0887, -0.1101, -0.0070,  0.0383,  0.0004]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [14]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [30]:
PATH='/content/gdrive/MyDrive/Colab Notebooks/fundusdataset/1000images/'

In [31]:
transform = transforms.Compose(
    [
     transforms.RandomRotation(45),
     transforms.RandomHorizontalFlip(),
     transforms.RandomCrop((1500,1500)),
     transforms.ToTensor(),
     
     
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [ ]:
a=torch.rand((1,3,384,384)).to(device)
model(a)

tensor([[-0.0125, -0.0010, -0.0864,  0.0090, -0.0817,  0.0209,  0.0551,  0.0471,
          0.0557, -0.0259,  0.1874, -0.0632,  0.0039, -0.0286, -0.0599,  0.0168,
          0.0617,  0.0070, -0.0242,  0.0189,  0.0119,  0.0359,  0.0332, -0.0477,
          0.0564,  0.0958, -0.0062, -0.1499, -0.1753,  0.0749, -0.0147,  0.0444,
          0.0997, -0.0755, -0.0031,  0.1158, -0.1942, -0.1274, -0.1188]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [32]:
fundus_data = torchvision.datasets.ImageFolder(root=PATH,transform=transform)


In [33]:
train_size=int(0.8*len(fundus_data))
test_size=len(fundus_data)-train_size

In [34]:
train_set, test_set=torch.utils.data.random_split(fundus_data,[train_size,test_size])

In [ ]:
torch.save(train_set,PATH+'trainset.csv')
torch.save(test_set,PATH+'testset.csv')

In [ ]:
train_set=torch.load(PATH+'trainset.csv')
test_set=torch.load(PATH+'testset.csv')

In [35]:
train_loader = torch.utils.data.DataLoader(train_set,drop_last=True,
                                          batch_size=1,
                                          shuffle=True,
                                          num_workers=2,
                                          )
test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=1,
                                          shuffle=False,
                                          num_workers=4,
                                          )

In [36]:
criterion = nn.CrossEntropyLoss().to(device)
#optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
optimizer=adabelief_pytorch.AdaBelief(model.parameters(),lr=0.001)
#lr_sche = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)


Please check your arguments if you have upgraded adabelief-pytorch from version 0.0.5.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  ---------
adabelief-pytorch=0.0.5  1e-08  False              False
>=0.1.0 (Current 0.2.0)  1e-16  True               True
SGD better than Adam (e.g. CNN for Image Classification)    Adam better than SGD (e.g. Transformer, GAN)
----------------------------------------------------------  ----------------------------------------------
Recommended eps = 1e-8                                      Recommended eps = 1e-16
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer
You can disable the log message by setting "print_change_log = False", though it is recommended to keep as a reminder.

Weight decoupling enabled in AdaBelief
Rectification enabled in AdaBelief


In [ ]:
print(len(train_loader))

800


In [37]:
trainedepochs=0
print(len(train_loader))

epochs = 500


for epoch in range(epochs):
  model.train()
  
  running_loss = 0.0

  for i, data in enumerate(train_loader, 0):
    # get the inputs
    inputs, labels = data
    inputs=inputs.to(device)
    labels = labels.to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs)

    
    
       # (numarray.max()/numarray[labels[0][0].to(torch.int).tolist()-1])
    #loss = (1/numnum[labels[0][0].to(torch.int).tolist()-1])*criterion(outputs, labels.to(torch.float)/10)
    loss = criterion(outputs, labels)
    
    #print( (numnum.max()/numnum[labels[0][0].to(torch.int).tolist()-1]))
    loss.backward()
    optimizer.step()

        # print statistics
    running_loss += loss.item()
    if i % 100 == 99:    
      print('[%d, %5d] loss: %.3f' %
            (epoch + 1, i + 1, running_loss / 20))
      running_loss = 0.0
    #lr_sche.step()
  
  if epoch % 50==49:
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
      for data in test_loader:
        
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        #print(predicted ,labels)
        correct += (predicted == labels).sum().item()

      print('Accuracy of the network on the {} test images: {} %%'.format(len(test_set),
                100 * correct / total))

        


  #if ( epoch==100 or epoch==150 or epoch==200 or epoch==250 or epoch==300 or epoch==350 or epoch==400 or epoch==450 or epoch==500 or epoch==550):
      
  if (epoch==100 or epoch==200 or epoch==300 or  epoch==400 or epoch==500 ):
    torch.save(optimizer.state_dict, PATH+'optimizer{}.pt'.format(trainedepochs+epoch))
    torch.save(model.state_dict,PATH+'model{}.pt'.format(trainedepochs+epoch))

        
        
      
      # loop over the dataset multiple times

    #Check Accuracy
    #acc = acc_check(resnet50, testloader, epoch, save=1)
    
    
torch.save(model.state_dict,PATH+'model{}.pt'.format(trainedepochs+epochs))
torch.save(optimizer.state_dict, PATH+'optimizer{}.pt'.format(trainedepochs+epochs))
print('Finished Training')

800


RuntimeError: ignored

In [ ]:
correct = 0
total = 0
model.eval() 
with torch.no_grad():
    for data in test_loader:
        
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
         
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        #print(predicted ,labels)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the {} test images: {} %%'.format(len(test_set),
    100 * correct / total))

Accuracy of the network on the 200 test images: 34.5 %%
